# Quadratic Programming

In [0]:
import numpy as np
import pandas as pd
import cvxpy as cp
from cvxopt import matrix
from cvxopt import solvers

## Example from the readings
Minimize the following formula:
$$\min\left[\frac{1}{2}\left(x_1^2-x_1x_2+2x_2^2\right)-3x_1-\frac{3}{2}x_2\right]$$

Such that:$\ \ \ -1\le x_1\le2\ \ \ \ \ \ \text{and}\ \ \ \ \ \ 0\le x_2\le3$

This is a QP that can be cast in standard form with:

$$H=\left(\begin{matrix}1&-\frac{1}{2}\\-\frac{1}{2}&2\end{matrix}\right)\ \ \ \ \ c^T=\left(\begin{matrix}-3&-\frac{3}{2}\end{matrix}\right)\ \ \ \ \ A=\left(\begin{matrix}1&0\\-1&0\\0&1\\0&-1\end{matrix}\right)\ \ \ \ \ b=\left(\begin{matrix}2\\1\\3\\0\end{matrix}\right)$$

The quadratic programming would be of the form:
$$\min\left[\frac{1}{2}x^THx+c^Tx\right]$$

In [64]:
# Define QP parameters (directly by columns)
H = matrix([[1, -.5], [-.5, 2]], tc='d')
c = matrix([-3, -1.5], tc='d')
A = matrix([[1,-1,0,0],[0,0,1,-1]], tc='d')
b = matrix([2,1,3,0], tc='d')

# Construct the QP, invoke solver
sol = solvers.qp(H, c, A, b)

# Extract optimal value and solution
sol['x']
sol['primal objective']

     pcost       dcost       gap    pres   dres
 0: -4.7164e+00 -1.4752e+01  1e+01  6e-17  1e-16
 1: -5.3811e+00 -5.8477e+00  5e-01  2e-16  1e-16
 2: -5.5593e+00 -5.5655e+00  6e-03  4e-16  2e-16
 3: -5.5625e+00 -5.5625e+00  6e-05  1e-16  9e-17
 4: -5.5625e+00 -5.5625e+00  6e-07  8e-17  6e-18
Optimal solution found.


-5.562499682594628

## Pre-Class Exercise
In this exercise, we will look at the problem of portfolio optimization, which we briefly investigated in lesson 1.1. We will now use a more realistic model, where we consider risk levels and constraints on minimum rates of return. Suppose that we have a vector $x∈R^N$, where the ith component $x_{i}$ represents the fraction of our capital that we have invested in asset i. We treat the rate of return of each asset as a random variable, where the mean rate of return is represented by a vector $µ ∈ R^N$ , and the covariance matrix for the rates of return over all assets is denoted $C ∈ R^{N×N}$ . One way to allocate assets is to minimize risk, subject to our portfolio making some minimum rate or return.

- Explain why the quadratic form $\frac{1}{2}x^TCx$ provides a measure of overall portfolio risk. This is what we want to minimize.

Diversifying the portfolio decreases the risk of loosing money if the assets are not correlated. In other words, we want to minimize the correlation between assets so that the risk embeded in one assest is not going to propagate through the whole portfolio.

- Let r denote the minimum rate of return for the portfolio. Explain why this translates into a constraint $\mu^Tx\ge r$

The vector $\mu$ represents the rate of return for all assets. The dot product of the vector with a given asset $x$ yeilds the expected return on that specific investment. Thus, r is the lower limit for the return for the portfolio.

- Explain why we additionally need the (element-wise) constraint x ≥ 0 and the equality constraint $\sum_i^{ }x_i=1$

The investments are expressed as a percentage growth rather than absolute values. In other words, the sum of percentages of each asset adds up to 100% and the proportion of each asset cannot be less than 0.

- A dataset of 225 different assets can be found here. The first line of the file tells us the number of assets (225). The next 225 lines list the mean rate of return and standard deviation for each of the 225 assets. The final 113 × 225 lines tell us the correlation between the rates of return of the different assets: the first and second column are two assets i and j, and the third column is the correlation between asset i and asset j. Note that only the upper triangle of this matrix is specified, since correlations must be symmetric.
- Load the data into Python (pre-processing if necessary) and create a vector µ for the mean rate of return, a vector σ for the standard deviations, and a matrix K for the correlations.
- Compute the covariance matrix C by using the identity $C_{ij}=K_{ij}\sigma_i\sigma_j$
- Using the cost and constraints described above, create and solve a quadratic program using CVXPY to find the optimal asset allocation, assuming a minimum return rate of 0.2%. Are there some assets in which we practically would not invest in this case?

In [0]:
file = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/port5.txt'

data = np.loadtxt(file, skiprows=1, max_rows=225)
data_cor = np.loadtxt(file, skiprows=226)

mu = data[:,0]
sigma = data[:,1]

K, CC = np.zeros((225, 225)), np.zeros((225, 225))
K[np.triu_indices(225, 0)] = data_cor[:,2]
K[np.tril_indices(225, -1)] = K.T[np.tril_indices(225, -1)]
C = sigma * (sigma * K).T
r = 0.002

In [61]:
# Optimization
x = cp.Variable(225)
objective = cp.Minimize((1/2)*cp.quad_form(x, C))
constant = [mu.T @ x >= r, cp.sum(x) == 1, x >= 0]
problem = cp.Problem(objective, constant)
print('Minimum risk: ', problem.solve())

Minimum risk:  0.00019491212566550894


In [60]:
X = pd.DataFrame(x.value, columns=['Risk'])
X.sort_values(by='Risk', ascending=False)[:10]

,Risk
61,0.256742
59,0.120080
195,0.098023
39,0.086598
42,0.081199
8,0.079523
128,0.074114
214,0.068842
96,0.059268
170,0.057275
